# Azure Machine Learning Model Deployment

In [6]:
%matplotlib inline
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
 
import azureml
from azureml.core import Workspace, Run, Dataset

# Display the core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.2.0


In [2]:
from azureml.core import Workspace
from azureml.core.model import Model
import os
ws = Workspace.from_config()
model = Model(ws, 'credit_card_model')

model.download(target_dir=os.getcwd(), exist_ok=True)

# verify the downloaded model file
file_path = os.path.join(os.getcwd(), "sklearn_mnist_model.pkl")

os.stat(file_path)

os.stat_result(st_mode=33279, st_ino=375, st_dev=46, st_nlink=1, st_uid=0, st_gid=0, st_size=1122, st_atime=1587123893, st_mtime=1587126053, st_ctime=1587126053)

In [3]:
import pickle
from sklearn.externals import joblib

clf = joblib.load(os.path.join(os.getcwd(), 'sklearn_mnist_model.pkl'))

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [4]:
clf

LogisticRegression(C=2.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=42, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [7]:
dataset = Dataset.get_by_name(ws, name='credit-card-data')
data = dataset.to_pandas_dataframe()

In [8]:
x_test = data.drop(['Class'], axis=1)

In [9]:
y_hat = clf.predict(x_test)

In [10]:
y_hat

array([0, 0, 0, ..., 0, 0, 0])

Two required functions in the scoring script:

1. The init() function, which typically loads the model into a global object. This function is run only once when the Docker container is started.

2. The run(input_data) function uses the model to predict a value based on the input data. Inputs and outputs to the run typically use JSON for serialization and de-serialization, but other formats are supported.

In [11]:
%%writefile score.py
import json
import numpy as np
import os
import pickle
from sklearn.externals import joblib
from sklearn.linear_model import LogisticRegression

from azureml.core.model import Model

def init():
    global model
    # retrieve the path to the model file using the model name
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'sklearn_mnist_model.pkl')
    model = joblib.load(model_path)

def run(raw_data):
    data = np.array(json.loads(raw_data)['data'])
    # make prediction
    y_hat = model.predict(data)
    # you can return any data type as long as it is JSON-serializable
    return y_hat.tolist()

Writing score.py


environment file, called myenv.yml, that specifies all of the script's package dependencies. This file is used to make sure that all of those dependencies are installed in the Docker image. This model needs scikit-learn and azureml-sdk. All custom environment files need to list azureml-defaults with verion >= 1.0.45 as a pip dependency. This package contains the functionality needed to host the model as a web service.

In [12]:
from azureml.core.conda_dependencies import CondaDependencies

myenv = CondaDependencies()
myenv.add_conda_package("scikit-learn")
myenv.add_pip_package("azureml-defaults")

with open("myenv.yml", "w") as f:
    f.write(myenv.serialize_to_string())

In [13]:
with open("myenv.yml", "r") as f:
    print(f.read())

# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
  - azureml-defaults
- scikit-learn
channels:
- anaconda
- conda-forge



Create a deployment configuration file. Specify the number of CPUs and gigabytes of RAM needed for your Container Instances container. Although it depends on your model, the default of one core and 1 gigabyte of RAM is sufficient for many models. If you need more later, you have to re-create the image and redeploy the service.

In [14]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={"data": "MNIST",  
                                                     "method": "sklearn"},
                                               description='Predict MNIST with sklearn')

# Deploy in Container Instances

Configure the image and deploy. The following code goes through these steps:

1. Build an image by using these files:
    - The scoring file, score.py.
    - The environment file, myenv.yml.
    - The model file.
2. Register the image under the workspace.
3. Send the image to the Container Instances container.
4. Start up a container in Container Instances by using the image.
5. Get the web service HTTP endpoint.

In [15]:
%%time
from azureml.core.webservice import Webservice
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment

myenv = Environment.from_conda_specification(name="myenv", file_path="myenv.yml")
inference_config = InferenceConfig(entry_script="score.py", environment=myenv)

service = Model.deploy(workspace=ws,
                       name='sklearn-mnist-svc',
                       models=[model], 
                       inference_config=inference_config,
                       deployment_config=aciconfig)

service.wait_for_deployment(show_output=True)

Running...................................................................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
CPU times: user 701 ms, sys: 65.9 ms, total: 767 ms
Wall time: 8min 42s


In [16]:
print(service.scoring_uri)

http://5ab1e38d-6642-4fa7-a6df-480036f3c5b6.eastus2.azurecontainer.io/score


In [28]:
import json

# find 30 random samples from test set
n = 30
sample_indices = np.random.permutation(x_test.shape[0])[0:n]

test_samples = json.dumps({"data": x_test.loc[sample_indices].values.tolist()})
test_samples = bytes(test_samples, encoding='utf8')

# predict using the deployed model
result = service.run(input_data=test_samples)

In [29]:
result

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]